In [1]:
library(lme4)
library(lmerTest)
library(emmeans)
library(repr)
library(glue)
library(performance)
library(IRdisplay)
library(comprehenr)
library(GLMMadaptive)
library(multilevelTools)
library(data.table)
library(dplyr)
library(ggplot2)
library(ordinal)
library(tibble)
library(tidyr)
library(data.table)
library(xtable)
library(RVAideMemoire)


Loading required package: Matrix


Attaching package: 'lmerTest'


The following object is masked from 'package:lme4':

    lmer


The following object is masked from 'package:stats':

    step



Attaching package: 'IRdisplay'


The following object is masked from 'package:performance':

    display



Attaching package: 'GLMMadaptive'


The following object is masked from 'package:lme4':

    negative.binomial



Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'ordinal'


The following object is masked from 'package:dplyr':

    slice



Attaching package: 'tidyr'


The following objects are masked from 'package:Matrix':

    expand, pack, unpack



Attaching package: 'xtable'


The following object is masked from 'package:IRdis

In [2]:
options(width = 160, repr.plot.width = 20, repr.plot.height = 8)

In [3]:
current_participants_df <- read.csv('./human_evals_data/current_participants_df.csv')

# Set columns as categorical
current_participants_df$participant_id <- as.factor(current_participants_df$participant_id)
current_participants_df$game_id <- as.factor(current_participants_df$game_id)
current_participants_df$game_type <- as.factor(current_participants_df$game_type)
current_participants_df$full_game_id <- as.factor(current_participants_df$full_game_id)

# Change the order of game_type levels
current_participants_df$game_type <- factor(current_participants_df$game_type, levels = c('unmatched', 'unmatched_top_30', 'matched', 'real'))


# Set columns as booleans
current_participants_df$real <- as.logical(current_participants_df$real)
current_participants_df$matched <- as.logical(current_participants_df$matched)
current_participants_df$model_game <- (!current_participants_df$real)

ATTRIBUTES <- c("confident", "fun_play", "fun_watch", "capability", "goldilocks", "creativity", "human_likeness")
NORMALIZED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("normalized_{attr}") } )
CENTERED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("centered_{attr}") } )
FULL_NORMALIZED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("full_normalized_{attr}") })

for (i in seq_along(CENTERED_ATTRIBUTES)) {
    current_participants_df[[CENTERED_ATTRIBUTES[i]]] <- current_participants_df[[ATTRIBUTES[i]]] - 3
}

for (i in seq_along(FULL_NORMALIZED_ATTRIBUTES)) {
    current_participants_df[[FULL_NORMALIZED_ATTRIBUTES[i]]] <- (current_participants_df[[ATTRIBUTES[i]]] - mean(current_participants_df[[ATTRIBUTES[i]]])) / sd(current_participants_df[[ATTRIBUTES[i]]])
}



print(dim(current_participants_df))
head(current_participants_df, 2)

[1] 1292   50


,participant_id,game_id,real,matched,confident,fun_play,fun_watch,capability,goldilocks,creativity,...,centered_goldilocks,centered_creativity,centered_human_likeness,full_normalized_confident,full_normalized_fun_play,full_normalized_fun_watch,full_normalized_capability,full_normalized_goldilocks,full_normalized_creativity,full_normalized_human_likeness
,<fct>,<fct>,<lgl>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 1, 4, 0, 2, 0, 0, 0, 1, 0, 0, 0)",FALSE,FALSE,4,3,3,5,3,2,...,0,-1,-1,0.3349299,0.685856,0.736462,1.7028050,0.3154651,-0.1941839,-0.3231405
2,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0)",FALSE,FALSE,4,3,3,4,3,3,...,0,0,-1,0.3349299,0.685856,0.736462,0.8894874,0.3154651,0.7489952,-0.3231405


In [4]:
filtered_df <- current_participants_df[current_participants_df$game_type != 'unmatched', ]
filtered_df$game_type <- droplevels(filtered_df$game_type)

print(dim(filtered_df))
head(filtered_df, 2)

[1] 892  50


,participant_id,game_id,real,matched,confident,fun_play,fun_watch,capability,goldilocks,creativity,...,centered_goldilocks,centered_creativity,centered_human_likeness,full_normalized_confident,full_normalized_fun_play,full_normalized_fun_watch,full_normalized_capability,full_normalized_goldilocks,full_normalized_creativity,full_normalized_human_likeness
,<fct>,<fct>,<lgl>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0)",TRUE,TRUE,4,2,2,5,4,3,...,1,0,-1,0.3349299,-0.2092207,-0.1445648,1.70280498,1.2524326,0.7489952,-0.3231405
4,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0)",FALSE,TRUE,5,4,4,3,2,2,...,-1,-1,-1,1.1049113,1.5809328,1.6174888,0.07616983,-0.6215025,-0.1941839,-0.3231405


In [5]:
# N_POINTS = 50


# buildFitnessData <- function() {
#     # Get the min and max of the normalized fitness
#     fitness.min <- min(filtered_df$normalized_fitness);
#     fitness.max <- max(filtered_df$normalized_fitness);
#     fitness.seq <- seq(fitness.min, fitness.max, length.out = N_POINTS);
#     # Create a data frame with a row for each sequence of fitness values for each game type
#     predict_data_df <- expand.grid(normalized_fitness = fitness.seq, game_type = levels(filtered_df$game_type));
#     # replicate each row 3 times (once for each percentile)
#     predict_data_df <- predict_data_df[rep(row.names(predict_data_df), each = 3),];

#     predict_data_df <- predict_data_df %>%
#         mutate(percentile = rep(c(20, 50, 80), times = N_POINTS * 3)) %>%
#         mutate(percentile = as.factor(percentile));

#     return(predict_data_df);
# }


# extractGameEffects <- function(model) {
#     if (missing(model)) {
#         model = attr.mm;
#     }

#     attr.game_effects_df <- ranef(model)$full_game_id;
#     colnames(attr.game_effects_df)[1] <- c("effect");
#     attr.game_effects_df <- rownames_to_column(attr.game_effects_df, var = "full_game_id");
#     attr.game_effects_df$full_game_id <- factor(attr.game_effects_df$full_game_id, levels = levels(filtered_df$full_game_id));

#     limited_filtered_df <- filtered_df %>%
#         select(full_game_id, game_type) %>%
#         distinct();

#     attr.game_effects_df <- left_join(attr.game_effects_df, limited_filtered_df, by = "full_game_id");


#     percentiles <- attr.game_effects_df %>%
#         group_by(game_type) %>%
#         summarize(percentile_20 = quantile(effect, 0.20),
#                 percentile_50 = quantile(effect, 0.50),
#                 percentile_80 = quantile(effect, 0.80)) %>%
#         pivot_longer(cols = starts_with("percentile"),
#                     names_to = "percentile",
#                     values_to = "game_effect") %>%
#         mutate(percentile = gsub("percentile_", "", percentile)) %>%
#         mutate(percentile = as.factor(percentile));

#     return(percentiles);
# }


# extractParticipantEffects <- function(model) {
#     if (missing(model)) {
#         model = attr.mm;
#     }

#     attr.participant_effects_df <- ranef(model)$participant_id;
#     colnames(attr.participant_effects_df)[1] <- c("effect");
#     attr.participant_effects_df <- rownames_to_column(attr.participant_effects_df, var = "participant_id");
#     attr.participant_effects_df$participant_id <- factor(attr.participant_effects_df$participant_id, levels = levels(filtered_df$participant_id));

#     percentiles <- attr.participant_effects_df %>%
#         summarize(percentile_20 = quantile(effect, 0.20),
#                 percentile_50 = quantile(effect, 0.50),
#                 percentile_80 = quantile(effect, 0.80)) %>%
#         pivot_longer(cols = starts_with("percentile"),
#                     names_to = "percentile",
#                     values_to = "participant_effect") %>%
#         mutate(percentile = gsub("percentile_", "", percentile)) %>%
#         mutate(percentile = as.factor(percentile)) %>%
#         rename(participant_percentile = percentile);

#     return(percentiles);
# }


# extractCoefData <- function(model) {
#     if (missing(model)) {
#         model = attr.mm;
#     }

#     attr.coef <- coef(attr.mm);
#     fitness_coef <- attr.coef[length(attr.coef)];
#     coef.df <- as.data.frame(attr.coef[5:(length(attr.coef) - 1)])
#     colnames(coef.df)[1] <- "game_type_intercept";
#     row_names <- rownames(coef.df);
#     row_names <- gsub("C\\(game_type\\)", "", row_names);
#     rownames(coef.df) <- row_names;

#     coef.df <- rownames_to_column(coef.df, var = "game_type");
#     coef.df <- rbind(coef.df, c(game_type = "unmatched_top_30", game_type_intercept = 0));
#     coef.df$game_type <- as.factor(coef.df$game_type);
#     coef.df$game_type_intercept <- as.numeric(coef.df$game_type_intercept);

#     return(coef.df);
# }

# plotPredictData <- function(merged_predict_df) {
#     participant_titles <- to_vec(for (p in levels(merged_predict_df$participant_percentile)) { glue("{p}th Percentile Participant") } );
#     names(participant_titles) <- levels(merged_predict_df$participant_percentile);

#     return(print(ggplot(merged_predict_df, aes(x = normalized_fitness, y = bin, color = game_type, linetype = percentile)) +
#         facet_wrap(vars(participant_percentile), nrow = 1, labeller = labeller(cyl = participant_titles)) +
#         geom_line(size = 1.25) +
#         labs(x = "Normalized Fitness", y = attr) +
#         scale_color_manual(values = c("unmatched" = "red", "unmatched_top_30" = "blue", "matched" = "green", "real" = "purple")) +
#         scale_linetype_manual(values = c("20" = "dashed", "50" = "solid", "80" = "dotted")) +
#         theme_minimal() +
#         theme(
#             plot.margin = margin(2, 2, 2, 2, "cm"),
#             text = element_text(size = 16),
#             axis.text = element_text(size = 14),
#         ) + 
#         ggtitle(glue("{attr} by Game Type and Percentile")) 
#         ));

#     # return(print(ggplot(merged_predict_df, aes(x = normalized_fitness, y = bin, color = game_type, linetype = percentile)) +
#     #     geom_line(size = 1.25) +
#     #     labs(x = "Normalized Fitness", y = attr) +
#     #     scale_color_manual(values = c("unmatched" = "red", "unmatched_top_30" = "blue", "matched" = "green", "real" = "purple")) +
#     #     scale_linetype_manual(values = c("20" = "dashed", "50" = "solid", "80" = "dotted")) +
#     #     theme_minimal() +
#     #     theme(
#     #         plot.margin = margin(2, 2, 2, 2, "cm"),
#     #         text = element_text(size = 16),
#     #         axis.text = element_text(size = 14),
#     #     ) + 
#     #     ggtitle(glue("{attr} by Game Type and Percentile"))))
# }

# plotModelPredictions <- function(model) {
#     if (missing(model)) {
#         model = attr.mm;
#     }

#     predict_data_df <- buildFitnessData();
#     percentiles_df <- extractGameEffects(model);

#     predict_data_df <-
#         left_join(predict_data_df, percentiles_df, by = c("game_type", "percentile"));
    
#     coef_df <- extractCoefData(model);
#     attr.coef <- coef(attr.mm);
#     fitness_coef <- attr.coef[length(attr.coef)];

#     participant_effects_df <- extractParticipantEffects(model);
#     predict_data_df <- merge(predict_data_df, participant_effects_df, by = NULL);

#     merged_predict_df <- 
#         merge(predict_data_df, coef_df, by = "game_type", all.x = TRUE) %>%
#         # add the prediction
#         mutate(prediction = (merged_predict_df$game_type_intercept + (fitness_coef * merged_predict_df$normalized_fitness) + merged_predict_df$game_effect + merged_predict_df$participant_effect)) %>%
#         # bin the prediction
#         mutate(bin = cut(prediction, breaks = c(-Inf, attr.coef[1:4], Inf), labels = 1:5, include.lowest = TRUE)) %>%
#         mutate(bin = as.double(bin)) %>%
#         # Add 0.1 if game type is 'matched' and 0.2 if game type is 'real'
#         mutate(bin = ifelse(game_type == 'unmatched_top_30', bin - 0.15, bin)) %>%
#         mutate(bin = ifelse(game_type == 'real', bin + 0.15, bin)) %>%
#         mutate(bin = ifelse(percentile == 20, bin - 0.05, bin)) %>%
#         mutate(bin = ifelse(percentile == 80, bin + 0.05, bin));

#     return(plotPredictData(merged_predict_df));
# }

## Do we need a random effect for participants?

In [13]:
# Iterate through attributes
for (attr in ATTRIBUTES) {
    attr.mm_no_participant_id <- clmm(
        formula = as.formula(glue("C({attr}) ~ 1 + C(game_type) + normalized_fitness + (1 | full_game_id)")),
        data = filtered_df, Hess = TRUE
    )
    attr.mm_with_participant_id <- clmm(
        formula = as.formula(glue("C({attr}) ~ 1 + C(game_type) + normalized_fitness + (1 | full_game_id) + (1 | participant_id)")),
        data = filtered_df, Hess = TRUE
    )

    anova_results <- anova(attr.mm_no_participant_id, attr.mm_with_participant_id)
    attr.aic_before <- anova_results$AIC[[1]]
    attr.aic_after <- anova_results$AIC[[2]]
    attr.aic_diff <- attr.aic_before - attr.aic_after
    attr.chisq <- anova_results$Chisq[[2]]
    attr.pvalue <- anova_results$`Pr(>Chisq)`[[2]] 
    
    print(glue("For {attr}, adding the participant ID random effects results in AIC diff: {format(attr.aic_diff, nsmall=3)} | Chi-squared: {format(attr.chisq, nsmall=3)} | p-value: {format(attr.pvalue, nsmall=3)}"))
}



For confident, adding the participant ID random effects results in AIC diff: 98.78088 | Chi-squared: NULL | p-value: 1.027435e-23
For fun_play, adding the participant ID random effects results in AIC diff: 248.459 | Chi-squared: NULL | p-value: 2.062364e-56
For fun_watch, adding the participant ID random effects results in AIC diff: 381.5017 | Chi-squared: NULL | p-value: 2.150804e-85
For capability, adding the participant ID random effects results in AIC diff: 350.6805 | Chi-squared: NULL | p-value: 1.10518e-78
For goldilocks, adding the participant ID random effects results in AIC diff: 89.72851 | Chi-squared: NULL | p-value: 9.942224e-22
For creativity, adding the participant ID random effects results in AIC diff: 257.707 | Chi-squared: NULL | p-value: 1.987821e-58
For human_likeness, adding the participant ID random effects results in AIC diff: 182.0796 | Chi-squared: NULL | p-value: 6.233415e-42


Absolutely we do.

## Do we need a random effect for games?

In [10]:


# Iterate through attributes
for (attr in ATTRIBUTES) {
    attr.mm_no_game_id <- clmm(
        formula = as.formula(glue("C({attr}) ~ 1 + C(game_type) + normalized_fitness + (1 | participant_id)")),
        data = filtered_df, Hess = TRUE
    )
    attr.mm_with_game_id <- clmm(
        formula = as.formula(glue("C({attr}) ~ 1 + C(game_type) + normalized_fitness + (1 | participant_id) + (1 | full_game_id)")),
        data = filtered_df, Hess = TRUE
    )

    anova_results <- anova(attr.mm_no_game_id, attr.mm_with_game_id)
    attr.aic_before <- anova_results$AIC[[1]]
    attr.aic_after <- anova_results$AIC[[2]]
    attr.aic_diff <- attr.aic_before - attr.aic_after
    attr.chisq <- anova_results$Chisq[[2]]
    attr.pvalue <- anova_results$`Pr(>Chisq)`[[2]] 
    
    print(glue("For {attr}, adding the game ID random effects results in AIC diff: {format(attr.aic_diff, nsmall=3)} | Chi-squared: {format(attr.chisq, nsmall=3)} | p-value: {format(attr.pvalue, nsmall=3)}"))
}



For confident, adding the game ID random effects results in AIC diff: 49.03556 | Chi-squared: NULL | p-value: 9.070776e-13
For fun_play, adding the game ID random effects results in AIC diff: 39.63944 | Chi-squared: NULL | p-value: 1.097565e-10
For fun_watch, adding the game ID random effects results in AIC diff: 26.17496 | Chi-squared: NULL | p-value: 1.108294e-07
For capability, adding the game ID random effects results in AIC diff: -1.977122 | Chi-squared: NULL | p-value: 0.8797742
For goldilocks, adding the game ID random effects results in AIC diff: 108.3868 | Chi-squared: NULL | p-value: 8.062017e-26
For creativity, adding the game ID random effects results in AIC diff: 53.75391 | Chi-squared: NULL | p-value: 8.213478e-14
For human_likeness, adding the game ID random effects results in AIC diff: 29.1491 | Chi-squared: NULL | p-value: 2.389503e-08


Sure seems to help in almost every attribute.

## Fit and analyze the full mixed effects model

In [15]:
analysis_df <- data.table::copy(filtered_df)
model_list <- list()

for (attr in ATTRIBUTES) {
    display_markdown(glue("## {attr}"))

    # Create formula with current attribute
    attr.mm <- clmm(
        formula = as.formula(glue("C({attr}) ~ 1  + normalized_fitness + C(game_type) + (1 | participant_id) + (1 | full_game_id)")),
        data = analysis_df, Hess = TRUE
    )

    model_list[[attr]] <- attr.mm

    display_markdown("### Mixed Effects Model")    
    print(summary(attr.mm))

    attr.marginal_means = emmeans(attr.mm, "game_type")
    display_markdown("### Marginal Means")
    print(pairs(attr.marginal_means, reverse = TRUE))

    display_markdown("------------------")
}


## confident

### Mixed Effects Model

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: C(confident) ~ 1 + normalized_fitness + C(game_type) + (1 | participant_id) +      (1 | full_game_id)
data:    analysis_df

 link  threshold nobs logLik   AIC     niter     max.grad cond.H 
 logit flexible  892  -1182.29 2382.59 755(3012) 2.29e-04 8.8e+01

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 1.2831   1.1327  
 full_game_id   (Intercept) 0.7669   0.8757  
Number of groups:  participant_id 100,  full_game_id 90 

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
normalized_fitness    0.8459     0.1504   5.625 1.86e-08 ***
C(game_type)matched   0.5253     0.2974   1.766   0.0773 .  
C(game_type)real      1.1511     0.2852   4.036 5.43e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
    Estimate Std. Error z value
1|2 -2.80972    0.26835 -10.471
2|3 -1.36745    0.24135  -5.666
3|4 -0.01

Warning message:
"contrasts dropped from factor C(game_type)"


### Marginal Means

...

 contrast                   estimate    SE  df z.ratio p.value
 matched - unmatched_top_30    0.551 0.310 Inf   1.776  0.1778
 real - unmatched_top_30       0.438 0.298 Inf   1.467  0.3067
 real - matched               -0.113 0.310 Inf  -0.364  0.9296

P value adjustment: tukey method for comparing a family of 3 estimates 
Cumulative Link Mixed Model fitted with the Laplace approximation

formula: C(human_likeness) ~ 1 + normalized_fitness + C(game_type) + (1 |      participant_id) + (1 | full_game_id)
data:    analysis_df

 link  threshold nobs logLik   AIC     niter     max.grad cond.H 
 logit flexible  892  -1215.28 2448.56 661(3291) 1.89e-03 7.5e+01

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 1.951    1.3967  
 full_game_id   (Intercept) 0.544    0.7375  
Number of groups:  participant_id 100,  full_game_id 90 

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
normalized_fitness    0.5698     0.1320   4.316 1.

Warning message:
"contrasts dropped from factor C(game_type)"


### Marginal Means

------------------

 contrast                   estimate    SE  df z.ratio p.value
 matched - unmatched_top_30    0.837 0.268 Inf   3.128  0.0050
 real - unmatched_top_30       1.446 0.258 Inf   5.597  <.0001
 real - matched                0.609 0.265 Inf   2.299  0.0559

P value adjustment: tukey method for comparing a family of 3 estimates 


# Mixed Effect Model Summary Table
This code generates Table SI-3, from which Table ED-1 is derived.

In [17]:
truncate_p_values <- FALSE
sig_figs <- 3

results_table <- data.frame(
    attribute = character(), 
    aic = numeric(),
    fit_est = numeric(),
    fit_z = numeric(),
    fit_p = numeric(),
    matched_est = numeric(),
    matched_z = numeric(),
    matched_p = numeric(),
    real_est = numeric(),
    real_z = numeric(),
    real_p = numeric()
)


for (name in names(model_list)) {
    model <- model_list[[name]]
    s <- summary(model)
    table_row <- data.frame(
        attribute = name,
        fit_est = glue("${formatC(s$coefficients[5, 1], digits = sig_figs, format = 'f')} \\pm {formatC(s$coefficients[5, 2], digits = sig_figs, format = 'f')}$"),  # s$coefficients[5, 1],
        fit_z = formatC(s$coefficients[5, 3], digits = sig_figs, format = 'f'),
        fit_p = s$coefficients[5, 4],
        matched_est = glue("${formatC(s$coefficients[6, 1], digits = sig_figs, format = 'f')} \\pm {formatC(s$coefficients[6, 2], digits = sig_figs, format = 'f')}$"),
        matched_z = formatC(s$coefficients[6, 3], digits = sig_figs, format = 'f'),
        matched_p = s$coefficients[6, 4],
        real_est = glue("${formatC(s$coefficients[7, 1], digits = sig_figs, format = 'f')} \\pm {formatC(s$coefficients[7, 2], digits = sig_figs, format = 'f')}$"),
        real_z = formatC(s$coefficients[7, 3], digits = sig_figs, format = 'f'),
        real_p = s$coefficients[7, 4]
    )
    results_table <- rbind(results_table, table_row)
}



# Iterate through the columns
for (col in c("fit_p", "matched_p", "real_p")) {
    # Iterate through the cells
    for (i in seq_len(nrow(results_table))) {
        # Get the current value
        value <- as.double(results_table[i, col])
        # Modify the value based on the conditions
        if (truncate_p_values && value < 10^-5) {
            results_table[i, col] <- "$P < \\num{1e-5}^{ *** }$"
        } else if (truncate_p_values && value < 10^-3) {
            results_table[i, col] <- "$P < \\num{1e-3}^{ *** }$"
        } else {
            use_scientific <- value < 0.01
            v <- formatC(value, digits = sig_figs, format = ifelse(use_scientific, "e", "f"))
            if (use_scientific) {
                v <- glue("\\num{{{v}}}")
            }
            stars <- ifelse(value < 0.001, "***",
                ifelse(value < 0.01, "**", ifelse(value < 0.05, "*", ""))
            )
            if (stars != "") {
                results_table[i, col] <- glue("$P = {v}^{{ {stars} }}$")
            } else {
                results_table[i, col] <- glue("$P = {v}$")
            }
        }
    }
}


results_table

attribute,fit_est,fit_z,fit_p,matched_est,matched_z,matched_p,real_est,real_z,real_p
<chr>,<glue>,<chr>,<chr>,<glue>,<chr>,<chr>,<glue>,<chr>,<chr>
confident,$0.846 \pm 0.150$,5.625,$P = \num{1.858e-08}^{ *** }$,$0.525 \pm 0.297$,1.766,$P = 0.077$,$1.151 \pm 0.285$,4.036,$P = \num{5.431e-05}^{ *** }$
fun_play,$0.396 \pm 0.135$,2.936,$P = \num{3.322e-03}^{ ** }$,$0.629 \pm 0.274$,2.298,$P = 0.022^{ * }$,$1.059 \pm 0.263$,4.021,$P = \num{5.797e-05}^{ *** }$
fun_watch,$0.191 \pm 0.130$,1.469,$P = 0.142$,$0.641 \pm 0.266$,2.414,$P = 0.016^{ * }$,$0.912 \pm 0.257$,3.547,$P = \num{3.901e-04}^{ *** }$
capability,$-0.189 \pm 0.087$,-2.163,$P = 0.031^{ * }$,$0.349 \pm 0.170$,2.048,$P = 0.041^{ * }$,$0.232 \pm 0.161$,1.441,$P = 0.150$
goldilocks,$-0.588 \pm 0.171$,-3.443,$P = \num{5.763e-04}^{ *** }$,$0.363 \pm 0.353$,1.029,$P = 0.304$,$-0.250 \pm 0.338$,-0.740,$P = 0.460$
creativity,$-0.486 \pm 0.152$,-3.191,$P = \num{1.416e-03}^{ ** }$,$0.551 \pm 0.310$,1.776,$P = 0.076$,$0.438 \pm 0.298$,1.467,$P = 0.142$
human_likeness,$0.570 \pm 0.132$,4.316,$P = \num{1.589e-05}^{ *** }$,$0.837 \pm 0.268$,3.128,$P = \num{1.762e-03}^{ ** }$,$1.446 \pm 0.258$,5.597,$P = \num{2.179e-08}^{ *** }$


In [18]:
# Convert the dataframe to an xtable object
table_latex <- xtable(results_table, digits = 3)

# Print the LaTeX table
print(table_latex, include.rownames = FALSE, floating = FALSE, sanitize.text.function = identity, digits=3)


% latex table generated in R 4.3.2 by xtable 1.8-4 package
% Wed May 29 14:22:27 2024
\begin{tabular}{llllllllll}
  \hline
attribute & fit_est & fit_z & fit_p & matched_est & matched_z & matched_p & real_est & real_z & real_p \\ 
  \hline
confident & $0.846 \pm 0.150$ & 5.625 & $P = \num{1.858e-08}^{ *** }$ & $0.525 \pm 0.297$ & 1.766 & $P = 0.077$ & $1.151 \pm 0.285$ & 4.036 & $P = \num{5.431e-05}^{ *** }$ \\ 
  fun_play & $0.396 \pm 0.135$ & 2.936 & $P = \num{3.322e-03}^{ ** }$ & $0.629 \pm 0.274$ & 2.298 & $P = 0.022^{ * }$ & $1.059 \pm 0.263$ & 4.021 & $P = \num{5.797e-05}^{ *** }$ \\ 
  fun_watch & $0.191 \pm 0.130$ & 1.469 & $P = 0.142$ & $0.641 \pm 0.266$ & 2.414 & $P = 0.016^{ * }$ & $0.912 \pm 0.257$ & 3.547 & $P = \num{3.901e-04}^{ *** }$ \\ 
  capability & $-0.189 \pm 0.087$ & -2.163 & $P = 0.031^{ * }$ & $0.349 \pm 0.170$ & 2.048 & $P = 0.041^{ * }$ & $0.232 \pm 0.161$ & 1.441 & $P = 0.150$ \\ 
  goldilocks & $-0.588 \pm 0.171$ & -3.443 & $P = \num{5.763e-04}^{ *** }$ & $0.

# Marginal Means Table
This code generates Table SI-4.

In [20]:
truncate_p_values <- FALSE
sig_figs <- 3

mm_results_table <- data.frame(
    attribute = character(),
    r_m_est = numeric(),
    r_m_z = numeric(),
    r_m_p = numeric(),
    r_u_est = numeric(),
    r_u_z = numeric(),
    r_u_p = numeric(),
    m_u_est = numeric(),
    m_u_z = numeric(),
    m_u_p = numeric()
    
)


for (name in names(model_list)) {
    model <- model_list[[name]]
    em <- emmeans(model, "game_type")
    p <- pairs(em, reverse = TRUE)
    s <- summary(p)
    table_row <- data.frame(
        attribute = name,
        r_m_est = glue("${formatC(s[3, 2], digits = sig_figs, format = 'f')} \\pm {formatC(s[3, 3], digits = sig_figs, format = 'f')}$"),
        r_m_z = formatC(s[3, 5], digits = sig_figs, format = "f"),
        r_m_p = s[3, 6],
        r_u_est = glue("${formatC(s[2, 2], digits = sig_figs, format = 'f')} \\pm {formatC(s[2, 3], digits = sig_figs, format = 'f')}$"),
        r_u_z = formatC(s[2, 5], digits = sig_figs, format = "f"),
        r_u_p = s[2, 6],
        m_u_est = glue("${formatC(s[1, 2], digits = sig_figs, format = 'f')} \\pm {formatC(s[1, 3], digits = sig_figs, format = 'f')}$"),
        m_u_z = formatC(s[1, 5], digits = sig_figs, format = "f"),
        m_u_p = s[1, 6]
    )
    mm_results_table <- rbind(mm_results_table, table_row)
}


# Iterate through the columns
for (col in c("m_u_p", "r_u_p", "r_m_p")) {
    # Iterate through the cells
    for (i in seq_len(nrow(mm_results_table))) {
        # Get the current value
        value <- as.double(mm_results_table[i, col])

        # Modify the value based on the conditions
        if (truncate_p_values && value < 10^-5) {
            mm_results_table[i, col] <- "$P < \\num{1e-5}^{ *** }$"
        } else if (truncate_p_values && value < 10^-3) {
            mm_results_table[i, col] <- "$P < \\num{1e-3}^{ *** }$"
        } else {
            use_scientific <- value < 0.01
            v <- formatC(value, digits = sig_figs, format = ifelse(use_scientific, "e", "f"))
            if (use_scientific) {
                v <- glue("\\num{{{v}}}")
            }
            stars <- ifelse(value < 0.001, "***",
                ifelse(value < 0.01, "**", ifelse(value < 0.05, "*", ""))
            )
            if (stars != "") {
                mm_results_table[i, col] <- glue("$P = {v}^{{ {stars} }}$")
            } else {
                mm_results_table[i, col] <- glue("$P = {v}$")
            }
        }
    }
}

mm_results_table

Warning message:
"contrasts dropped from factor C(game_type)"


Warning message:
"contrasts dropped from factor C(game_type)"
Warning message:
"contrasts dropped from factor C(game_type)"
Warning message:
"contrasts dropped from factor C(game_type)"
Warning message:
"contrasts dropped from factor C(game_type)"
Warning message:
"contrasts dropped from factor C(game_type)"
Warning message:
"contrasts dropped from factor C(game_type)"


attribute,r_m_est,r_m_z,r_m_p,r_u_est,r_u_z,r_u_p,m_u_est,m_u_z,m_u_p
<chr>,<glue>,<chr>,<chr>,<glue>,<chr>,<chr>,<glue>,<chr>,<chr>
confident,$0.626 \pm 0.305$,2.055,$P = 0.099$,$1.151 \pm 0.285$,4.036,$P = \num{1.606e-04}^{ *** }$,$0.525 \pm 0.297$,1.766,$P = 0.181$
fun_play,$0.430 \pm 0.273$,1.577,$P = 0.256$,$1.059 \pm 0.263$,4.021,$P = \num{1.713e-04}^{ *** }$,$0.629 \pm 0.274$,2.298,$P = 0.056$
fun_watch,$0.271 \pm 0.264$,1.025,$P = 0.561$,$0.912 \pm 0.257$,3.547,$P = \num{1.135e-03}^{ ** }$,$0.641 \pm 0.266$,2.414,$P = 0.042^{ * }$
capability,$-0.117 \pm 0.167$,-0.701,$P = 0.763$,$0.232 \pm 0.161$,1.441,$P = 0.320$,$0.349 \pm 0.170$,2.048,$P = 0.101$
goldilocks,$-0.613 \pm 0.355$,-1.725,$P = 0.196$,$-0.250 \pm 0.338$,-0.740,$P = 0.740$,$0.363 \pm 0.353$,1.029,$P = 0.559$
creativity,$-0.113 \pm 0.310$,-0.364,$P = 0.930$,$0.438 \pm 0.298$,1.467,$P = 0.307$,$0.551 \pm 0.310$,1.776,$P = 0.178$
human_likeness,$0.609 \pm 0.265$,2.299,$P = 0.056$,$1.446 \pm 0.258$,5.597,$P = \num{6.531e-08}^{ *** }$,$0.837 \pm 0.268$,3.128,$P = \num{5.013e-03}^{ ** }$


In [21]:

# Convert the dataframe to an xtable object
table_latex <- xtable(mm_results_table, digits = 3)

# Print the LaTeX table
print(table_latex, include.rownames = FALSE, floating = FALSE, sanitize.text.function = identity, digits=3)



% latex table generated in R 4.3.2 by xtable 1.8-4 package
% Wed May 29 14:23:36 2024
\begin{tabular}{llllllllll}
  \hline
attribute & r_m_est & r_m_z & r_m_p & r_u_est & r_u_z & r_u_p & m_u_est & m_u_z & m_u_p \\ 
  \hline
confident & $0.626 \pm 0.305$ & 2.055 & $P = 0.099$ & $1.151 \pm 0.285$ & 4.036 & $P = \num{1.606e-04}^{ *** }$ & $0.525 \pm 0.297$ & 1.766 & $P = 0.181$ \\ 
  fun_play & $0.430 \pm 0.273$ & 1.577 & $P = 0.256$ & $1.059 \pm 0.263$ & 4.021 & $P = \num{1.713e-04}^{ *** }$ & $0.629 \pm 0.274$ & 2.298 & $P = 0.056$ \\ 
  fun_watch & $0.271 \pm 0.264$ & 1.025 & $P = 0.561$ & $0.912 \pm 0.257$ & 3.547 & $P = \num{1.135e-03}^{ ** }$ & $0.641 \pm 0.266$ & 2.414 & $P = 0.042^{ * }$ \\ 
  capability & $-0.117 \pm 0.167$ & -0.701 & $P = 0.763$ & $0.232 \pm 0.161$ & 1.441 & $P = 0.320$ & $0.349 \pm 0.170$ & 2.048 & $P = 0.101$ \\ 
  goldilocks & $-0.613 \pm 0.355$ & -1.725 & $P = 0.196$ & $-0.250 \pm 0.338$ & -0.740 & $P = 0.740$ & $0.363 \pm 0.353$ & 1.029 & $P = 0.559$ \\ 
  